#### 【 데이터 합치기 - merge】
- 기준에 따라 두 Series/DataFrame 병합/합치하는 것
- 기준이 되는 열이름/행인덱스 => Key
- SQL의 join 기능
- 관계형 병합

[1] 모듈 로딩 <hr>

In [37]:
## 모듈 로딩
import pandas as pd 

[2] 데이터 합치기<hr>

In [38]:
## ========================================================================
## 함   수   pandas.merge(left, right, how, on, left_on, right_on, ... )
## ========================================================================
## 매개변수
##     * left                       : 왼쪽 DataFrame/Series
##     * right                      : 오른쪽 DataFrame/Series
##     * how=‘inner’,               : 병합 방식
##     * on=None,                   : None -> 공통 열 키로 설정
##     * left_on,                   : 왼쪽에서 사용할 열 이름
##     * right_on,                  : 오른쪽에서 사용할 열 이름
##     * left_index=False,          : 결합 기준으로 왼쪽 인덱스 사용 여부
##     * right_index=False,         : 결합 기준으로 오른쪽 인덱스 사용 여부
##     * sort=False,                : 결과를 키 기준으로 정렬 여부
##     * suffixes=('_x', '_y'),     : 중복 열 이름 뒤에 붙일 접미사  
## =================================================================

>>> [2-1] DataFrame 합치기

In [ ]:
## DataFrame 인스턴스 생성
df1 = pd.DataFrame({'lkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': [1, 2, 3, 5]})

df2 = pd.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': [5, 6, 7, 8]}, 
                    index=[2,3,4,5])

display(df1, df2)

,lkey,value
0,foo,1
1,bar,2
2,baz,3
3,foo,5


,rkey,value
2,foo,5
3,bar,6
4,baz,7
5,foo,8


In [40]:
## ------------------------------------------
## 열방향 교집합 합치기
## - 기준 키 컬럼 미지정 => 동일 컬럼명 기준 
## ------------------------------------------
dataDF = pd.merge(df1, df2, how='inner')

display( dataDF )

,lkey,value,rkey
0,foo,5,foo


In [42]:
## ------------------------------------------
## 열방향 합집합 합치기
## - 기준 키 컬럼 미지정 => 동일 컬럼명 기준 
## ------------------------------------------
dataDF = pd.merge(df1, df2, how='outer')
display( dataDF )

,lkey,value,rkey
0,foo,1,NaN
1,bar,2,NaN
2,baz,3,NaN
3,foo,5,foo
4,NaN,6,bar
5,NaN,7,baz
6,NaN,8,foo


In [43]:
## ------------------------------------------
## 열방향 왼쪽 기준 합치기
## - 기준 키 컬럼 미지정 => 동일 컬럼명 기준 
## ------------------------------------------
dataDF = pd.merge(df1, df2, how='left')
display( dataDF )

,lkey,value,rkey
0,foo,1,NaN
1,bar,2,NaN
2,baz,3,NaN
3,foo,5,foo


In [44]:
## ------------------------------------------
## 열방향 오른쪽 기준 합치기
## - 기준 키 컬럼 미지정 => 동일 컬럼명 기준 
## ------------------------------------------
dataDF = pd.merge(df1, df2, how='right')
display( dataDF )

,lkey,value,rkey
0,foo,5,foo
1,NaN,6,bar
2,NaN,7,baz
3,NaN,8,foo


In [48]:
## ------------------------------------------
## 행 방향 / 교집합
## - 존재하지 않는 열/컬럼 삭제
## ------------------------------------------
dataDF = pd.merge(df1, df2, left_on='lkey', right_on='rkey')
display( dataDF )

,lkey,value_x,rkey,value_y
0,foo,1,foo,5
1,foo,1,foo,8
2,bar,2,bar,6
3,baz,3,baz,7
4,foo,5,foo,5
5,foo,5,foo,8


In [50]:
## ------------------------------------------
## 행 방향 / 교집합
## - 존재하지 않는 열/컬럼 삭제
## ------------------------------------------
dataDF = pd.merge(df1, df2, left_index=True,  right_index=True)
display( dataDF )

,lkey,value_x,rkey,value_y
2,baz,3,foo,5
3,foo,5,bar,6


>>> [2-2] Series 합치기

In [54]:
## ============================================
## Series → DataFrame 변환 후 인덱스 기준 병합
## ============================================
## Series 인스턴스 생성
s1 = pd.Series([10, 20, 30], index=['a', 'b', 'c'], name='X')
s2 = pd.Series([100, 200, 300], index=['b', 'c', 'd'], name='Y')

## Series 병합
result = pd.merge(s1.to_frame(), 
                  s2.to_frame(),
                  left_index=True, 
                  right_index=True,
                  how='outer')


display(result)

,X,Y
a,10.0,NaN
b,20.0,100.0
c,30.0,200.0
d,NaN,300.0


In [57]:
## ============================================
## Series → DataFrame 변환 : 공통 열/컬럼 설정 
## ============================================
s1 = pd.Series(['foo', 'bar', 'baz'], name='key')
s2 = pd.Series([10, 20, 30], name='value')

df1 = s1.to_frame()
df2 = s2.to_frame()

# 공통 열 이름 지정
df1['value'] = [1, 2, 3]  # 왼쪽에 값 추가
df2['key'] = ['foo', 'baz', 'bar']

# key를 기준으로 병합
result = pd.merge(df1, df2, on='key', how='inner')

display(result)

,key,value_x,value_y
0,foo,1,10
1,bar,2,30
2,baz,3,20


In [56]:
## ============================================
## Series , DataFrame 병합 
## ============================================
df = pd.DataFrame({'A': [1, 2, 3]}, index=['a', 'b', 'c'])
sr = pd.Series([10, 20, 30], index=['b', 'c', 'd'], name='B')

# Series를 오른쪽 DF로 변환
result = pd.merge(df, sr.to_frame(), 
                  left_index=True, 
                  right_index=True, 
                  how='outer')

display(result)

,A,B
a,1.0,NaN
b,2.0,10.0
c,3.0,20.0
d,NaN,30.0
